# TTS & voice dubbing

### *How to dub a video voice?*

- Create a function that merges generated audio on video

In [ ]:
from moviepy.editor import *

# voice dubbing in the new language
def add_audio_on_video(translated_audio, video_title):
    
    source_video = f"/workspace/ai-multimedia-translator/video_samples/{video_title}.mp4"
    
    videoclip = VideoFileClip(source_video)
    audioclip = AudioFileClip(translated_audio)

    new_audioclip = CompositeAudioClip([audioclip])
    new_videoclip = f"/workspace/ai-multimedia-translator/outputs/{video_title}.mp4"
    
    videoclip.audio = new_audioclip
    videoclip.write_videofile(new_videoclip)
    
    return new_videoclip

- Get audio transcription from **TTS** model

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('/workspace/.env')
load_dotenv(dotenv_path=dotenv_path)

# access the environment variables from the .env file
tts_endpoint_url = os.environ.get('TTS_DE_DE_ENDPOINT')
ai_endpoint_token = os.environ.get('OVH_AI_ENDPOINTS_ACCESS_TOKEN')

In [ ]:
import riva.client
from pydub import AudioSegment

# TTS function
def tts_transcription(output_nmt, video_title, voice_type, target_lang):
    
    # connect with riva asr server
    tts_service = riva.client.SpeechSynthesisService(
                    riva.client.Auth(
                        uri=tts_endpoint_url,
                        use_ssl=True, 
                        metadata_args=[["authorization", f"bearer {ai_endpoint_token}"]]
                    )
                )
    
    # set up tts config
    sample_rate_hz = 16000
    req = { 
            "language_code"  : f"{target_lang}",
            "encoding"       : riva.client.AudioEncoding.LINEAR_PCM ,  
            "sample_rate_hz" : sample_rate_hz,                       
            "voice_name"     : f"{voice_type}"                        
    }

    output_audio = 0
    output_audio_file = f"/workspace/ai-multimedia-translator/outputs/{target_lang[:-3]}_{video_title}.wav"
    for i in range(len(output_nmt)):
        
        # add silence between audio sample
        if i==0:
            duration_silence = output_nmt[i][1]
        else:
            duration_silence = output_nmt[i][1] - output_nmt[i-1][2]
        silent_segment = AudioSegment.silent(duration = duration_silence)
        output_audio += silent_segment
        
        # create tts transcription
        req["text"] = output_nmt[i][0]
        resp = tts_service.synthesize(**req)
        sound_segment = AudioSegment(
            resp.audio,
            sample_width=2,
            frame_rate=16000,
            channels=1,
        )
        print(type(resp.audio))
        output_audio += sound_segment
    
    # export new audio as wav file
    output_audio.export(output_audio_file, format="wav")
    
    # add new voice on video
    voice_dubbing = add_audio_on_video(output_audio_file, video_title)
    
    return voice_dubbing

<img src="images/audio_silence_generation_riva.png"  width=800 />

- Play input video sample

In [ ]:
from IPython.display import Video

video_input = "video_ovhcloud_en_1.mp4"
Video(f"/workspace/ai-multimedia-translator/video_samples/{video_input}", embed=True)

- Translation output from **RIVA NMT** model

In [ ]:
# text translation
output_nmt = [
    ['Ist Ihr Domainname derzeit bei einem anderen Registrar und Sie möchten ihn zu Ovh Cloud übertragen? Dies können Sie über ein Transferverfahren tun.', 
     1080, 
     8040], 
    ['Bevor Sie beginnen, sehen Sie sich die Liste der Voraussetzungen in der Beschreibung dieses Videos an.', 
     12320, 
     17240], 
    ['Es ist perfekt. Lass uns anfangen.', 
     18160, 
     20200], 
    ['1. Es ist wichtig, dass die mit dem Domainnamen verbundenen Informationen aktuell sind. \
    Falls nicht, kontaktieren Sie bitte den aktuellen Domainnamen-Registrar und korrigieren Sie diese.', 
     21320, 
     31080], 
    ['Der zweite Schritt ist die Entsperrung des Domainnamens. Dieser Vorgang wird durch den aktuellen Domainnamen-Registrar durchgeführt.', 
     33280, 
     39960], 
    ['Wenn Sie nicht sicher sind, wie Sie diesen Schritt ausführen sollen, wenden Sie sich bitte an das Kundensupport-Team, \
    um Unterstützung bei diesem Schritt zu erhalten.', 
     40800, 
     46680], 
    ['Sobald Sie freigeschaltet sind, erhalten Sie einen Transfercode.', 
     49040, 
     51600], 
    ['Dieser Code wird manchmal auch als verschiedene Namen bezeichnet, wie Transfer Code, Auth Code, Auth Information oder Epp Code.', 
     52600, 
     60240], 
    ['Wenn Sie diesen Code haben, können Sie die ⁇ bertragung Ihres Domainnamens auf Ovh Cloud beantragen. \
    Der Vorgang ist einfach: Sie müssen ihn auf unserer Website bestellen. Das System erkennt, dass der Domainname existiert, und bietet an, \
    ihn zu übertragen. Nach den Validierungsschritten werden Sie aufgefordert, Ihren Transfercode anzugeben. Geben Sie diesen in das entsprechende Feld ein.', 
     61240, 
     80520], 
    ['Sie erhalten dann eine Bestellung, die ⁇ berweisung beginnt erst, wenn die Zahlung erfolgt ist.', 
     81920, 
     86920], 
    ['Von diesem Punkt, Sie werden in der Lage sein, den Transfer Fortschritt über die Ovh Cloud Control Panel verfolgen.', 
     87720, 
     92760], 
    ['Um dies zu tun, können Sie den Abschnitt Web Cloud aufrufen.', 
     93640,
     96080], 
    ['Klicken Sie auf den betreffenden Domainnamen und dann auf die laufende Operation.', 
     96920, 
     100800], 
    ['Sobald die Bestellung bezahlt wurde, erhält der Domainnamen-Registrar eine Validierungsanfrage, woraufhin der Transfer beginnt.', 
     101880, 
     108800], 
    ['Die Operation kann durchschnittlich fünf Tage dauern.', 
     109840, 
     112240], 
    ['Wenn der Registrar eine Ablehnung ausstellt, wenden Sie sich bitte an das Kundensupport-Team, um herauszufinden, warum.', 
     113080, 
     118800], 
    ['Sie müssen dann die ⁇ bertragungsanforderung aus dem Tracking Space neu starten.', 
     119640, 
     123000], 
    ['Wenn das Verfahren reibungslos verläuft, wird Ihr Domainname nun über Ovh Cloud verwaltet und Sie können ihn über das Ovh Cloud Control Panel verwalten.', 
     124160, 
     131200], 
    ['Und natürlich, wenn Sie mehr über das Thema in diesem Video erkunden möchten, lesen Sie bitte unsere Anleitungen in der Beschreibung.', 
     132000, 
     138520]
]
print("Translation output - RIVA NMT:\n\n", output_nmt)

- Get results from **RIVA TTS**

In [ ]:
text_to_audio = tts_transcription(output_nmt, video_input[:-4], "German-DE-Male-1", "de-DE")
text_to_audio

- Play the resulting video

In [ ]:
from IPython.display import Video

Video(f"/workspace/ai-multimedia-translator/outputs/{video_input}", embed=True)